# Data Cleaning
## Procedure:
1. Import raw csvs
2. Remove extraneous columns
3. Convert streak information into integers
4. Create a dictionary of dataframes
5. Once dictionary is complete, save all csvs

In [ ]:
import pandas as pd
def get_teams():
    teams = [item.strip() for item in open("all_teams.txt",'r')]
    return teams

get_teams()

In [ ]:
def create_truncated_csv(teams):
    #takes a list of team names, opens the cooresponding files, truncates the data and outputs a new file
    for team in teams:    
            print(f'TeamResults\\{team.strip()}_2019.csv')
            table = pd.read_csv(f'TeamResults\\csv\\{team.strip()}_2019.csv')
            new = table.drop(["Date", "Unnamed: 2", "R", "RA", "Inn", "Rank", "GB", "Win", "Loss", "Save", "Time", "D/N", "Attendance", "Orig. Scheduled" ], axis=1)
            new.to_csv(f'TeamResults\\{team.strip()}_2019_truncated.csv', index=False, encoding='utf8')
#create_truncated_csv(get_teams())

### Convert streak data into integer values

In [ ]:
def convert_streak_to_int(col):
    """
    takes a string or list of streak data in the form of '-' & '+' signs and outputs in integer format.  It is assumed that there will be no mixing of + & -
    return string or list of ints representing the win streak at that time
    """
    if isinstance(col,str):
        streak = [col]
    elif isinstance(col,list):
        streak = col
    else:
        raise Exception(f"parameter given was of {type(col)} when it should be string or list")
    
    streak_int = []
    for record in streak:
        streak = len(record)
        if record[0] =='-':
            streak = -streak
        streak_int.append(streak)    
        
    if len(col) == 1:
        return streak_int[0]
    else:
        return streak_int
    
def change_column(df, col, new):
    #parameters: dataframe, column name, list of new values
    df[col] = new
    return df

def home_away_game_list_maker(col):
    """
    Convert a column from NaN and '@' symbols into 1/0 for home or away"""
    if isinstance(col,list):
        loc = []
    else:
        raise Exception(f"parameter given was of {type(col)} when it should be string or list")
    for item in col:
        if item == "@":
            loc.append(0)
        else:
            loc.append(1)
    return loc                     
 

"""
def create_int_streak():
    #This function need only be run once
    for team in get_teams():
        t = pd.read_csv(f'TeamResults\\{team.strip()}_2019_truncated.csv')
        new = convert_streak_to_int(list(t["Streak"]))
        t2 = change_column(t,"Streak",new)
        t2.to_csv(f'TeamResults\\new\\{team.strip()}_2019_truncated.csv', index=False)
"""

In [ ]:
season['PHI']["Home_Game"]

In [ ]:
for k,v in season.items():
    #season[k] = v.rename(columns={"Unnamed: 4": 'Home_Game'})
    #print(v)
    converted = home_away_game_list_maker(v["Home_Game"].tolist())
    #print(converted)
    change_column(v,"Home_Game",converted)
    #print(df)



## Create a dictionary of all teams

In [ ]:
def create_team_dictionary():
    season = {}
    teams = get_teams()
    for team in teams:
        t = pd.read_csv(f'TeamResults\\all_truncated_csv_WL\\{team.strip()}_2019_truncated.csv')
        season[team.strip()] = t
    return season

season = create_team_dictionary()
season

In [ ]:
def remove_first_column():
    for team in get_teams():
        t = pd.read_csv(f'TeamResults\\{team.strip()}_2019_truncated.csv')
        print(t.drop(t.iloc[:,0:1],axis=1))
        t.drop(t.iloc[:,0:1], axis=1).to_csv(f'TeamResults\\{team.strip()}_2019_truncated.csv', index=False)
        #t.to_csv(f'TeamResults\\{team.strip()}_2019_truncated.csv')

### Convert some date values into win-loss records

In [ ]:
def convert_date_to_record(col):
    import re
    new = []
    for i in range(len(col)):
        found = re.search(r'([A-Z])\w+',col[i])
        if found:
            win = text_to_num(found.group(0))            
            loss = re.search(r'(\d+)',col[i]).group(0)
            print(loss)
            if loss == '00':
                loss = '0'
            new.append(f'{win}-{loss}')
        if col[i][-2:] == '00':
            new.append(f'{col[i][:-2]}{0}')
        else:
            new.append(col[i])

    return new

def remove_double_zeros():
    return 0
def text_to_num(text):
    switcher = {
     "Jan" : 1,
     "Feb" : 2,
     "Mar" : 3,
     "Apr" : 4,
     "May" : 5,
     "Jun" : 6,
     "Jul" : 7,
     "Aug" : 8,
     "Sep" : 9,
      "Oct": 10,
      "Nov": 11,
      "Dec": 12
    }
    return switcher[text]
def remove_all_but_first_char(col):
    new = []
    for i in range(len(col)):
        new.append(col[i][0])
    return new

def remove_date_values(dictionary):
    for team,df in dictionary.items():
        new = convert_date_to_record(df["W-L"])
        dictionary[team] = change_column(df,"W-L",new)

    
def clean_W_L(dictionary):
    for team,df in dictionary.items():
        new = remove_all_but_first_char(df["W/L"])
        dictionary[team] = change_column(df,"W/L",new)

    
#remove_date_values(season)
clean_W_L(season)



In [84]:
def save_season_to_csv(season):
    for k,v in season.items():
        v.to_csv(f'TeamResults\\all_truncated_csv_WL\\{k.strip()}_2019_truncated.csv', index=False)
    

In [85]:
save_season_to_csv(season)

In [ ]:
season['SEA']["Home_Game"].tolist()

In [ ]:
def convert_WL_Percent(rec):
    """
    Parameter: string or list containg W-L data, separated by a hyphen
    Return: float(s) containing win percentage(s)"""
    if isinstance(rec,str):
        rec = [rec]
    elif isinstance(rec,list):
        rec = rec
    else:
        raise Exception(f"parameter given was of {type(rec)} when it should be string or list")
    
    percent = []
    for record in rec:
        record = record.split('-')
        win = float(record[0])
        loss = float(record[1])
        total = win + loss
        percent.append(win / total)
        
    if len(percent) == 1:
        return percent[0]
    else:
        return percent

def convert_WL_Int(result):
    """
    Parameter: takes a string or list of "W" or "L" data and converts that to 1 and 0
    This is essentially a custom labeller
    returns list of outcomes in int form"""
    if isinstance(result,str):
        results = [result]
    elif isinstance(result,list):
        results = result
    else:
        raise Exception(f"parameter given was of {type(result)} when it should be string or list")
    
    outcomes = []
    for result in results:
        if result =="W":
            outcomes.append(1)
        else:
            outcomes.append(0)

    if len(outcomes) == 1:
        return outcomes[0]
    else:
        return outcomes

In [ ]:
record = [item for item in season["SEA"]["W-L"]]
convert_WL_Percent(record)

In [ ]:
import numpy as np
result = [item for item in season["SEA"]["W/L"]]
np.array(convert_WL_Int(result))

In [ ]:
def getProbOfWin(season, team):
#calculate both teams winning percentage
#which team has the higher percentage?  Did they win?  Counting.
#always stick with the home team
#outcomes: home higher - win: W++, home higher - loss: L++, home lower - win: L++, home lower - loss:W++
    T = 0
    F = 0
    for index,item in season[team].iterrows():
        if item.loc["Opp"] not in season:
            continue
        homeTeamWinPercentage = convert_WL_Percent(item.loc["W-L"])
        awayTeamWinPercentage = convert_WL_Percent(season[item.loc["Opp"]].loc[index]["W-L"])
        gameResult = convert_WL_Int(item.loc["W/L"])
        
        if homeTeamWinPercentage >= awayTeamWinPercentage:
            if gameResult == 1:
                T += 1
            else:
                F += 1
        else:
            if gameResult == 1:
                F += 1
            else:
                T += 1   
    
    
    return(T,F)

In [ ]:
probability = {}
for team in get_teams():
    win,loss = getProbOfWin(season,team)
    probability[team] = (convert_WL_Percent(season[team].iloc[-1]["W-L"]),win / (win+loss))
    
probability
    

In [ ]:
import matplotlib.pyplot as plt
X_values = []
y_values = []
for key, value in probability.items():
    X_values.append(value[0])
    y_values.append(value[1])

plt.figure(figsize= (20,10))
plt.title("Probability best team wins vs team record")
plt.ylabel("Probability best team wins")
plt.xlabel("2019 Season winning percentage")

plt.scatter(X_values,y_values,color='r')
plt.show()